In [ ]:
import pandas as pd
import numpy as np

import torch
from torch import nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from preprocess import one_hot, one_hot_cut, one_hot_qcut, margin_prob, margin_rank

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv('train.csv')
x1, y1, _ = margin_prob(df)
x_train, x_test, y_train, y_test = train_test_split(x1, y1, test_size=0.25, random_state=2)

input_size = x1.shape[1]
hidden_sizes = [256, 64]
output_size = 2
# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1)).cuda()

x_train_tensor = torch.tensor(x_train.values.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.to_numpy().T).to(device)

x_test_tensor = torch.tensor(x_test.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.to_numpy().T).to(device)

learning_rate = 1e-4
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(6000):
    y_pred = model(x_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor)
    if t % 600 == 0:
        print(t, loss.item())
        y_val = model(x_test_tensor)
        y_val = np.argmax(y_val.detach().cpu().numpy(), axis=1)
        
        acc = accuracy_score(y_test_tensor.cpu().numpy(), y_val)
        f1=f1_score(y_test_tensor.cpu().numpy(), y_val, average='weighted')
        print("acc: %.2f%%" % (acc * 100.0))
        print("f1: %.2f%%" % (f1 * 100.0))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()